# A notebook to email geocomp grads the kata solutions

This is based on my [email from python](Email_from_python.ipynb) notebook which is based on [Corey Schafer's video](https://www.youtube.com/watch?v=JRCJ6RtE3xU&t=751s).

For the Google API, I used both [Google's docs](https://developers.google.com/sheets/api/guides/values) and [this medium article](https://medium.com/analytics-vidhya/how-to-read-and-write-data-to-google-spreadsheet-using-python-ebf54d51a72c) by [Prafulla Dalvi](https://medium.com/@prafuld3).

In [ ]:
#email imports
import smtplib
from email.message import EmailMessage
from enviro import GMAIL_LOGIN, GMAIL_PWD, SPREADSHEET_ID, SPREADSHEET_ID_TESTING, PROJECT_PATH

#Google API imports
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

#General imports
import glob
import pandas as pd
from datetime import date

In [ ]:
### GOOGLE API GLOBALS
# If modifying these scopes, delete the file token.pickle and rerun pickle creation functions.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# set up variables
login = GMAIL_LOGIN
password = GMAIL_PWD

In [ ]:
def connect_to_google_sheets_API():
    """
    Connect to the Google Sheet API, verify credentials and return a service object.
    """
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
            
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)
    
    return service


def read_from_google_sheet(service, spreadsheet_id):
    """Read data from the google sheet using the Sheets API"""
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id, range='Sheet1').execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found, aborting.')
        return None
    else:
        df = pd.DataFrame.from_records(values, columns=values[0])
        df = df.drop([0])
        df = df.reset_index(drop=True)
    
    return df


def write_to_google_sheet(df, service, spreadsheet_id):
    """Write date to Google Sheet log file for given class_abr and challenge"""
    response_date = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id,
                                                           valueInputOption='RAW',
                                                           range='Sheet1',
                                                           body=dict(
                                                               majorDimension='ROWS',
                                                               values=df.T.reset_index().T.values.tolist()
                                                           )
                                                          ).execute()
    print('Google sheet successfully updated')
    
    return None


def get_recipients(class_abr, spreadsheet_id):
    """
    Given a class abreviation, return a dict of names and emails.
    args:
        class_abr: `str` class abreviation for example: `apr20` or `mar20`
    """
    df = read_from_google_sheet(connect_to_google_sheets_API(), spreadsheet_id)
    df = df.loc[df['class_abr'] == class_abr]
    
    if len(df['class_abr'] == class_abr) == 0:
        print('The `class_abr` you provided does not exist or was misspelled.')
        return None
    
    return pd.Series(df.email.values, index=df.recipient).to_dict()


def get_solution(challenge, project_path):
    """Given a challenge name, return the corresponding path to the solution file."""
    challenge = (challenge.split('-')[0].title() + '-' + challenge.split('-')[1] 
                 if '-' in challenge else challenge.title())
    
    try:
        solution = glob.glob(f'{project_path}{challenge}_solution.ipynb')[0]
    except IndexError:
        print('The challenge you provided does not exist or was misspelled.')
        return None
    
    return solution


def make_df(class_abr, challenge, date, spreadsheet_id):
    """Given a class abreviation, challenge and date, return, make df from google sheet"""
    df = read_from_google_sheet(connect_to_google_sheets_API(), spreadsheet_id)
    column = get_solution(challenge, PROJECT_PATH).split('/')[-1]
    df.loc[df['class_abr'] == class_abr, column] = str(date)
    df = df.fillna('')
    
    return df

In [ ]:
MESSAGE = """Dear {}, 

I am enclosing the solutions to the {} challenge. I would recommend that you only use this if you're really stuck, or, once you've finished, to see some insights included in the solution.

You can get the next challenge by completing this one !

I'd also like to point you towards Matt's blog about this: 
https://agilescientific.com/blog/2020/4/16/geoscientist-challenge-thyself

and remember you can also get help here https://help.agilescientific.com/home

Finally we'd love to hear your feedback about these challenges. Are they fun? are they useful? are they hard enough or too hard?

And of course, you can unsubscribe at any time by replying to me.

Best wishes,
Rob

Robert Leckenby, PhD
Geoscientist at Agile*
agilescientific.com

PS: this email was sent with Python, because why not ;-)
    """

In [ ]:
def send_kata_solution(challenge, recipient, msg_to, msg_from='robert@agilescientific.com'):
    """
    Send an email with the kata solution as an attachment to the recipient, by name.
    
    args:
        challenge: `str` name of the kata challenge
        recipient: `str` name of the recipient
        msg_to: `str` valid email address of the `recipient` to send the message to
    kwargs:
        msg_from: `str` email from which to send the email - this is only for the reply-to field.
            The email is actually sent from my personal-work email
    Returns:
        None
    """
    msg = EmailMessage()
    msg['Subject'] = f'kata.geosci solution for challenge: {challenge}'
    msg['From'] = msg_from
    msg['To'] = msg_to
    msg['Cc'] = msg_from
    msg.set_content(MESSAGE.format(recipient, challenge))
    
    with open(get_solution(challenge, PROJECT_PATH), 'rb') as f:
        file_data = f.read()
        file_name = f.name.split('/')[-1]
    msg.add_attachment(file_data, maintype='text', subtype='ipynb', filename=file_name)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(login, password)
        # *************** WARNING ***************
        # the next line sends the actual email
        smtp.send_message(msg)
        
    # logging
    print(f'On {date.today()} the *{challenge}* challenge was sent to:\n{msg_to}\n')
    
    return

In [ ]:
def send_emails(class_abr, challenge, project_path, run_case=0):
    """
    Sends emails to students with a given challenge. Provides provision for manual tests.
    args:
        class_abr: `str` class abreviation of type "apr20" or "mar20" for example
        challenge: `str` kata challenge name
        project_path: `str` path to kata solutions folder
    kwargs:
        run_case: `int` from `set({0, 1, 99})` where:
            0: Dry run only, simply lists out names, emails and challenges for recipients in this class_abr, 
               does not write to Google sheet
            1: this sends actual emails for each email in class_abr with attached 
               solution and *DOES WRITE TO* Google sheet TESTING FILE
            99: *NOT A TEST*, this sends actual emails for each email in class_abr with attached 
               solution and *DOES WRITE TO* Google sheet PRODUCTION FILE (`99` rather than `2` to avoid typos)
    """
    if run_case == 0:
        print('Dry run only. Emails would have been sent to these recipients at these emails:\n')
        for recipient, email in get_recipients(class_abr, spreadsheet_id).items():
            print(f'recipient: {recipient}\temail: {email}\tchallenge: {challenge}')
        print('\nEnd.')
        return
    
    elif run_case == 1:
        spreadsheet_id = SPREADSHEET_ID_TESTING
    
    elif run_case == 99:
        spreadsheet_id = SPREADSHEET_ID
    
    else:
        print('Incorrect `run_case` kwarg passed, `run_case` must be `int` from `set({0, 1, 99})`')
        return
        
    if (get_recipients(class_abr, spreadsheet_id) == None) or (get_solution(challenge, PROJECT_PATH) == None):
        print('aborting.')
        return

    for recipient, email in get_recipients(class_abr, spreadsheet_id).items():
        send_kata_solution(challenge, recipient, email)

    df = make_df(class_abr, challenge, date.today(), spreadsheet_id)
    service = connect_to_google_sheets_API()

    write_to_google_sheet(df, service, spreadsheet_id)

    print(f'Check sending email account ({login}) for errors.\n')
    print('Done =============================================\n')

    return

#### SEND KATA EMAILS

In [ ]:
send_emails('Shell_Hou', 'sequence', PROJECT_PATH, run_case=0)